In [33]:
import torch
from torch.distributions import Categorical
import einops
from math import log

In [34]:
batch_size = 3
folds = 4
samples = 5
seq_len = 10
K = 37  # number of categories

In [35]:
t = torch.rand(batch_size * folds)


logits = torch.rand(batch_size * folds * samples, seq_len, K, requires_grad=True)
# logits = (logits - 0.5) * 1e8

cat = Categorical(logits=(logits - 0.5) * 1e18)

entropy = cat.entropy()  # should be shape (batch_size * folds * samples, seq_len)

expected_entropy = einops.reduce(
    entropy,
    "(batch_size folds samples) seq_len -> (batch_size folds)",
    "mean",
    batch_size=batch_size,
    folds=folds,
    samples=samples,
)

target_entropy = log(37) * (1 - t)

# return F.mse_loss(expected_entropy, target_entropy, reduction="mean")

In [36]:
mse = torch.mean((expected_entropy - target_entropy) ** 2)

In [37]:
mse

tensor(2.7897, grad_fn=<MeanBackward0>)

In [38]:
mse.backward()

In [39]:
logits.grad

tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        ...,

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0., 